In [1]:
import numpy as np
import math
from plotly import tools
import plotly.plotly as py
import plotly.graph_objs as go

class AdaptiveFilter():
    def __init__(self, eta=0.01, delay = 5, epochs=50, max_error=10**-6):
        self.eta = eta
        self.epochs = epochs
        self.delay = delay
        self.max_error = max_error
    def train(self, data, target):
        len_data = len(data)
        data = np.array(data)
        target = np.array(target)
        self.w = np.zeros(self.delay)
        self.b = 0
        self.errors = []
        for _ in range(self.epochs):
            error = 0
            for n in range(0, len_data - self.delay):
                data_n = data[n:self.delay + n]
                error_n = target[n + self.delay] - self.predict(data_n)
                self.w = self.w + self.eta * error_n * data_n
                self.b = self.b + self.eta * error_n
                error += error_n**2
            error = math.sqrt(error/len_data)
            if (error < self.max_error):
                break
            self.errors.append(error)
        return self

    def net_input(self, X):
        return np.dot(X, self.w) + self.b

    def predict(self, X):
        return self.net_input(np.array(X))
    
def create_data1(input_signal, t, h):
    grid = np.arange(t[0],t[1]+h, h)
    x_real = np.array([input_signal(ti) for ti in  grid])
    return grid, x_real
def create_data3(input_signal,out_signal, t, h):
    grid = np.arange(t[0],t[1]+h, h)
    x_real = np.array([input_signal(ti) for ti in  grid])
    y_real = np.array([out_signal(ti) for ti in  grid])
    return grid, x_real, y_real
    
def predict_data1(net, x_real, delay, t, h):
    grid  = np.arange(t[0]+ h*delay,t[1]+h, h)
    x_predict = []
    for n in range(0, len(x_real) - delay):
        x_predict.append(net.predict(x_real[n:n+delay]))
    x_predict = np.array(x_predict)
    return grid, x_predict
def predict_data2(net, input_signal, delay, t, h, shift):
    new_t = t
    new_t[0] = t[0] + h*shift
    new_t[1] = t[1] + h*shift
    grid  = np.arange(new_t[0],new_t[1]+h, h)
    x_real = [input_signal(ti) for ti in  grid]
    x_predict = x_real[:delay]
    for n in range(0, len(grid) - delay):
        x_predict.append(net.predict(x_predict[n:delay + n]))
    x_real = np.array(x_real)
    x_predict = np.array(x_predict)
    return grid, x_real, x_predict

    
def predict_data3(net, x_real, delay, t, h):
    grid  = np.arange(t[0]+ h*delay,t[1]+h, h)
    y_predict = []
    for n in range(0, len(x_real) - delay):
        y_predict.append(net.predict(x_real[n:n+delay]))
    y_predict = np.array(y_predict)
    return grid, y_predict



In [20]:
input_signal = lambda t: math.sin(-2*t**2 + 7*t) - 1/2*math.sin(t)
t = [0,4.5]
h = 0.025
delay = 5
data0 = create_data1(input_signal, t, h)
net = AdaptiveFilter(epochs = 600, delay = delay)
net.train(data0[1], data0[1])
data1 = predict_data1(net, data0[1], delay, t, h)
data2 = predict_data2(net, input_signal, delay, t, h, 180)
input_signal = lambda t: math.sin(t**2 - 6*t + 3) 
out_signal = lambda t: 1/3*math.sin(t**2 - 6*t - math.pi/6)
t = [0, 6]
net2 = AdaptiveFilter(epochs = 600, delay = delay)
data3 = create_data3(input_signal, out_signal, t, h)
net2.train(data3[1],data3[2])
data4 = predict_data3(net2, data3[1], delay, t, h)




In [21]:
trace0 = go.Scatter(
    x=data0[0],
    y=data0[1],
    name='Real X',
    mode = 'lines'
)
trace1 = go.Scatter(
    x=data1[0],
    y=data1[1],
    name='Predict X',
    mode = 'lines'
)
trace2 = go.Scatter(
    x=[i for i in range(1, len(net.errors)+1)],
    y=net.errors,
    name='Error',
    mode = 'lines'
)
fig = tools.make_subplots(rows=2, cols=1, subplot_titles=('One-step predict', 'Errors by step'))
fig.append_trace(trace0, 1, 1)
fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 2, 1)
fig['layout']['xaxis2'].update(title='step')
fig['layout']['yaxis2'].update(title='error')
py.iplot(fig, filename='customizing-subplot-axes')



This is the format of your plot grid:
[ (1,1) x1,y1 ]
[ (2,1) x2,y2 ]



In [22]:
trace3 = go.Scatter(
    x=data2[0],
    y=data2[2],
    name='Predict X',
    mode = 'lines'
)
trace4 = go.Scatter(
    x=data2[0],
    y=data2[1],
    name='Real X',
    mode = 'lines'
)
data = [trace3, trace4]
layout = go.Layout(
    title='Multi-step prediction')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename = 'basic-lined', )

In [5]:
trace5 = go.Scatter(
    x=data3[0],
    y=data3[1],
    name='Real X',
    mode = 'lines'
)
trace6 = go.Scatter(
    x=data3[0],
    y=data3[2],
    name='Real Y',
    mode = 'lines'
)
trace7 = go.Scatter(
    x=data4[0],
    y=data4[1],
    name='Predict Y',
    mode = 'lines'
)
data = [trace5, trace6, trace7]
layout = go.Layout(
    title='Adaptive filter')
fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename = 'basic-lined', )